In [1]:
%reset -f

In [2]:
# (Paso 1 - Celda 1) CARGA Y RENOMBRADO INICIAL

import pandas as pd
import numpy as np
from datetime import datetime

# ¡IMPORTANTE! Asegúrate que tu archivo se llame 'datos_encuesta.csv'
file_name = "datos_encuesta.csv" 

try:
    df = pd.read_csv(file_name, sep=';', low_memory=False) 
    print(f"Dataset cargado correctamente. Filas: {len(df)}")
except FileNotFoundError:
    print(f"ERROR CRÍTICO: El archivo '{file_name}' no se encuentra.")
    exit()

# Columnas vacías remanentes y de encabezado a eliminar
columns_to_drop = [
    'PERFIL', 'COMPETENCIAS_IA', 'PERTINENCIA_LABORAL', 'EVALUACION_DOCENTE',
    'CONFIANZA_LABORAL', '_validation_status', '_notes', '_submitted_by', '_tags'
]

# Diccionario de Renombre: Preguntas Largas a Variables Cortas
column_rename_map = {
    # Perfil / Demográficas
    '¿En qué universidad estudias actualmente?': 'universidad_original',
    '¿En qué país estás cursando tus estudios?': 'pais',
    '¿A qué área principal pertenece tu carrera?': 'area_carrera',
    'Por favor, escribe el nombre de tu carrera o programa de estudios.': 'nombre_carrera',
    '¿Qué semestre estás cursando actualmente?': 'semestre',
    '¿Cuál es tu edad?': 'edad',
    '¿Con qué género te identificas?': 'genero',
    # Competencias Digitales y Blandas
    'Análisis e interpretación de datos.': 'comp_analisis_datos',
    'Fundamentos de ciberseguridad.': 'comp_ciberseguridad',
    'Conceptos básicos de programación y automatización.': 'comp_programacion',
    'Manejo de herramientas de IA (ej. ChatGPT, Copilot, Gemini, DALL-E)': 'comp_manejo_ia',
    'Pensamiento crítico y resolución de problemas complejos': 'comp_pensamiento_critico',
    'Comunicación efectiva en entornos digitales': 'comp_comunicacion_digital',
    'Adaptabilidad y aprendizaje continuo': 'comp_adaptabilidad',
    'Creatividad e innovación': 'comp_creatividad',
    'Colaboración en equipos multidisciplinarios': 'comp_colaboracion',
    # Pertinencia Laboral
    'El currículo de mi carrera está alineado con las habilidades demandadas por el mercado laboral digital.': 'pert_curriculo_alineado',
    'Mi universidad promueve activamente el desarrollo de competencias digitales y blandas.': 'pert_promocion_digital',
    'Siento que la formación que he recibido me prepara para trabajos que aún no existen.': 'pert_preparacion_trabajos_futuros',
    'El uso de la tecnología y la IA está bien integrado en las asignaturas de mi carrera.': 'pert_integracion_ia_asignaturas',
    'La metodología de enseñanza en mi carrera fomenta la adaptabilidad y el aprendizaje continuo.': 'pert_fomento_adaptabilidad',
    # Confianza Laboral
    'Me siento seguro(a) de que mi formación académica me prepara para conseguir un empleo relevante en mi área.': 'conf_seguridad_empleo',
    'Mi perfil de egreso es competitivo en el mercado laboral actual.': 'conf_perfil_competitivo',
    'Considero que mis habilidades blandas (ej. comunicación, creatividad) me dan una ventaja en el mercado laboral.': 'conf_habilidades_blandas_ventaja',
    'Creo que mi conocimiento sobre herramientas de IA mejorará mis oportunidades laborales.': 'conf_conocimiento_ia_oportunidad',
    'Habilidades técnicas y digitales (ej. programación, análisis de datos)': 'conf_habilidades_tecnicas_imp',
    'Habilidades blandas (ej. pensamiento crítico, adaptabilidad)': 'conf_habilidades_blandas_imp',
    'Experiencia práctica o pasantías': 'conf_experiencia_practica_imp',
    'Red de contactos (networking)': 'conf_networking_imp',
    'Título académico de la universidad': 'conf_titulo_imp',
    # Evaluación Docente / Rol del Profesorado
    'El profesorado de mi carrera está capacitado para integrar herramientas de IA en la enseñanza.': 'eval_docente_capacitado_ia',
    'Mi universidad fomenta la innovación en las metodologías de enseñanza para adaptarse a la era digital.': 'eval_universidad_fomenta_innovacion',
    'Mi universidad tiene los recursos adecuados (plataformas, software) para formar profesionales en la era digital.': 'eval_universidad_recursos_adecuados',
    'En general, recomendaría mi universidad como una institución que prepara para los desafíos del mercado laboral digital.': 'eval_recomendacion_general',
    'El rol principal del profesorado será el de facilitador y guía del aprendizaje.': 'rol_profesor_facilitador',
    'El profesorado deberá enfocarse más en enseñar habilidades humanas (pensamiento crítico, ética, creatividad) que en transmitir información.': 'rol_profesor_habilidades_humanas',
    'El rol del profesorado se reducirá significativamente, siendo reemplazado en gran medida por la IA.': 'rol_profesor_reemplazo_ia',
    # Metadatos de KoboToolbox
    '_id': 'kobo_id', '_uuid': 'kobo_uuid', '_submission_time': 'fecha_registro',
    '_status': 'kobo_status', '__version__': 'kobo_version', '_index': 'kobo_index',
}

# Aplicar renombrado y eliminación de columnas
df_clean = df.drop(columns=columns_to_drop, errors='ignore').rename(columns=column_rename_map)

print("\nPrimeras 5 columnas del DataFrame limpio:")
print(df_clean.head().iloc[:, 0:5].to_markdown(index=False))

Dataset cargado correctamente. Filas: 455

Primeras 5 columnas del DataFrame limpio:
| universidad_original                              | pais     | area_carrera                          | nombre_carrera                    | semestre                 |
|:--------------------------------------------------|:---------|:--------------------------------------|:----------------------------------|:-------------------------|
| Instituto Tecnológico de Tuxtla Gutiérrez Chiapas | México   | Ciencias Económicas y Administrativas | Ingeniería en Gestión Empresarial | 9no Semestre             |
| universidad de envigago                           | Colombia | Ciencias Sociales y Humanidades       | Derecho                           | 8vo Semestre             |
| IUE                                               | Colombia | Ciencias Sociales y Humanidades       | Ciencias Juridicas                | 9no Semestre             |
| Institución Universitaria de Envigado             | Colombia | Ciencias S

In [3]:
# (Paso 2 - Celda 2) LIMPIEZA DE UNIVERSIDADES

# 1. Normalizar a minúsculas y quitar espacios en blanco
df_clean['universidad_limpia'] = df_clean['universidad_original'].astype(str).str.lower().str.strip()

# 2. Definir el diccionario de mapeo para estandarizar las principales universidades
mapping = {
    'espoch': 'ESCUELA SUPERIOR POLITÉCNICA DE CHIMBORAZO (ESPOCH)',
    'escuela superior politécnica de chimborazo': 'ESCUELA SUPERIOR POLITÉCNICA DE CHIMBORAZO (ESPOCH)',
    'unicach': 'UNIVERSIDAD DE CIENCIAS Y ARTES DE CHIAPAS (UNICACH)',
    'universidad de ciencias y artes de chiapas': 'UNIVERSIDAD DE CIENCIAS Y ARTES DE CHIAPAS (UNICACH)',
    'iue': 'INSTITUCIÓN UNIVERSITARIA DE ENVIGADO (IUE)',
    'institución universitaria de envigado': 'INSTITUCIÓN UNIVERSITARIA DE ENVIGADO (IUE)',
    'universidad de envigado': 'INSTITUCIÓN UNIVERSITARIA DE ENVIGADO (IUE)',
    'universidad de envigago': 'INSTITUCIÓN UNIVERSITARIA DE ENVIGADO (IUE)',
    'uniminuto': 'CORPORACIÓN UNIVERSITARIA MINUTO DE DIOS (UNIMINUTO)',
    'minuto de dios': 'CORPORACIÓN UNIVERSITARIA MINUTO DE DIOS (UNIMINUTO)',
}

# 3. Aplicar el mapeo
def clean_university(name):
    return mapping.get(name, name.title())

df_clean['universidad_limpia'] = df_clean['universidad_limpia'].apply(clean_university)

# Eliminamos la columna original ya que tenemos la limpia
df_clean = df_clean.drop(columns=['universidad_original'], errors='ignore')

print("Las 5 universidades más frecuentes después de la limpieza:")
print(df_clean['universidad_limpia'].value_counts().head(5).to_markdown())

Las 5 universidades más frecuentes después de la limpieza:
| universidad_limpia                                   |   count |
|:-----------------------------------------------------|--------:|
| ESCUELA SUPERIOR POLITÉCNICA DE CHIMBORAZO (ESPOCH)  |     152 |
| INSTITUCIÓN UNIVERSITARIA DE ENVIGADO (IUE)          |     121 |
| UNIVERSIDAD DE CIENCIAS Y ARTES DE CHIAPAS (UNICACH) |     103 |
| CORPORACIÓN UNIVERSITARIA MINUTO DE DIOS (UNIMINUTO) |      29 |
| Institucion Universitaria De Envigado                |       4 |


In [4]:
# (Paso 3 - Celda 3) LIMPIEZA DE EDAD Y SEMESTRE (CÓDIGO FINAL CORREGIDO)

import re # Necesario para buscar los números en el texto del semestre
from datetime import datetime # Necesario para el cálculo de edad
import numpy as np # Necesaria para manejar valores NaN

# --- Limpieza de EDAD (Convierte fechas y texto a número) ---
reference_date = datetime(2025, 10, 30) 

def calculate_age(value):
    if pd.isna(value) or value is None: return np.nan
    value_str = str(value).strip()
    try: return int(value_str) 
    except ValueError: pass
    try:
        birth_date = datetime.strptime(value_str, '%d/%m/%Y')
        age = reference_date.year - birth_date.year - ((reference_date.month, reference_date.day) < (birth_date.month, birth_date.day))
        return age
    except ValueError: return np.nan 

df_clean['edad_num'] = df_clean['edad'].apply(calculate_age)
df_clean = df_clean.drop(columns=['edad'], errors='ignore')


# --- Limpieza de SEMESTRE (CÓDIGO CORREGIDO: Extrae solo los dígitos) ---
def clean_semestre_corrected(semestre):
    if pd.isna(semestre): return np.nan
    s = str(semestre)
    # Busca la primera secuencia de dígitos (números) en el texto (ej: de "9no Semestre" extrae "9")
    match = re.search(r'(\d+)', s)
    if match:
        return int(match.group(1))
    return np.nan

df_clean['semestre_num'] = df_clean['semestre'].apply(clean_semestre_corrected)
df_clean = df_clean.drop(columns=['semestre'], errors='ignore')

print("Estadísticas de la edad numérica y el semestre numérico (debe tener 455 conteos):")
print(df_clean[['edad_num', 'semestre_num']].describe().to_markdown())

Estadísticas de la edad numérica y el semestre numérico (debe tener 455 conteos):
|       |   edad_num |   semestre_num |
|:------|-----------:|---------------:|
| count |   455      |     455        |
| mean  |    23.7714 |       8.17143  |
| std   |     5.8694 |       0.451694 |
| min   |    17      |       8        |
| 25%   |    21      |       8        |
| 50%   |    22      |       8        |
| 75%   |    24      |       8        |
| max   |    65      |      10        |


In [5]:
# (Paso 4 - Celda 4) CODIFICACIÓN LIKERT Y PREPARACIÓN FINAL

# --- 1. Definición de Escalas Likert ---
agreement_scale = {
    'Totalmente de acuerdo': 5, 'De acuerdo': 4, 'Neutral': 3, 'En desacuerdo': 2, 'Totalmente en desacuerdo': 1
}
skill_scale = {
    'Avanzado': 4, 'Intermedio': 3, 'Básico': 2, 'Ninguno': 1
}
importance_scale = {
    'Muy importante': 5, 'Importante': 4, 'Neutral': 3, 'Poca importancia': 2, 'Nada importante': 1
}

# --- 2. Aplicación de la Codificación (Crea las nuevas columnas con sufijo _num) ---
# Columnas de Pertinencia, Confianza (acuerdo) y Evaluación Docente
agreement_cols = [col for col in df_clean.columns if col.startswith(('pert_', 'conf_seguridad_', 'conf_perfil_', 'conf_habilidades_blandas_ventaja', 'conf_conocimiento_', 'eval_', 'rol_profesor_'))]
for col in agreement_cols: df_clean[col + '_num'] = df_clean[col].map(agreement_scale)

# Columnas de Competencias Digitales (Nivel de Habilidad)
skill_cols = [col for col in df_clean.columns if col.startswith('comp_')]
for col in skill_cols: df_clean[col + '_num'] = df_clean[col].map(skill_scale)

# Columnas de Importancia de Factores
importance_cols = [col for col in df_clean.columns if col.endswith('_imp')]
for col in importance_cols: df_clean[col + '_num'] = df_clean[col].map(importance_scale)

# --- 3. Limpieza final del DataFrame (Crea df_final) ---
# Mantenemos solo las categóricas clave y todas las nuevas *_num
cols_to_keep_original = ['universidad_limpia', 'pais', 'area_carrera', 'nombre_carrera', 'genero', 'fecha_registro']
cols_to_drop = [col for col in df_clean.columns if col not in cols_to_keep_original and not col.endswith('_num')]
df_final = df_clean.drop(columns=cols_to_drop, errors='ignore')

print("El DataFrame final 'df_final' está listo para el análisis KDD.")
print(df_final.info())

El DataFrame final 'df_final' está listo para el análisis KDD.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 38 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   pais                                     455 non-null    object 
 1   area_carrera                             455 non-null    object 
 2   nombre_carrera                           455 non-null    object 
 3   genero                                   455 non-null    object 
 4   fecha_registro                           455 non-null    object 
 5   universidad_limpia                       455 non-null    object 
 6   edad_num                                 455 non-null    int64  
 7   semestre_num                             455 non-null    int64  
 8   pert_curriculo_alineado_num              455 non-null    int64  
 9   pert_promocion_digital_num               455 non-null    

In [6]:
# (Paso 5 - Celda 5) DIAGNÓSTICO AVANZADO: PROMEDIOS POR PAÍS

# 1. Calcular el promedio de cada grupo de indicadores por fila (encuestado)
# Nota: Pandas calcula el promedio ignorando automáticamente los NaN (datos faltantes).
df_final['COMPETENCIAS_IA_Promedio'] = df_final[[col for col in df_final.columns if col.startswith('comp_') and col.endswith('_num')]].mean(axis=1)
df_final['PERTINENCIA_LABORAL_Promedio'] = df_final[[col for col in df_final.columns if col.startswith('pert_') and col.endswith('_num')]].mean(axis=1)
df_final['CONFIANZA_LABORAL_Promedio'] = df_final[[col for col in df_final.columns if col.startswith('conf_') and col.endswith('_num') and not col.endswith('_imp_num')]].mean(axis=1)
df_final['IMPORTANCIA_FACTORES_Promedio'] = df_final[[col for col in df_final.columns if col.startswith('conf_') and col.endswith('_imp_num')]].mean(axis=1)
df_final['EVALUACIÓN_DOCENTE_Promedio'] = df_final[[col for col in df_final.columns if col.startswith('eval_') and col.endswith('_num')]].mean(axis=1)
df_final['ROL_PROFESORADO_Promedio'] = df_final[[col for col in df_final.columns if col.startswith('rol_profesor_') and col.endswith('_num')]].mean(axis=1)

# 2. Agrupar por País y calcular el promedio final de cada indicador
avg_cols = [col for col in df_final.columns if col.endswith('_Promedio')]
descriptive_by_country = df_final.groupby('pais')[avg_cols].mean().reset_index()

# 3. Preparar la tabla para el BI y exportar
descriptive_by_country_melted = descriptive_by_country.melt(id_vars='pais', var_name='Indicador', value_name='Promedio')
descriptive_by_country_melted['Promedio'] = descriptive_by_country_melted['Promedio'].round(3)

csv_file_name = "analisis_promedios_por_pais.csv"
descriptive_by_country_melted.to_csv(csv_file_name, index=False)

print("--- DIAGNÓSTICO AVANZADO: PROMEDIOS POR PAÍS ---")
print("Tabla de Promedios por Indicador (en escala 1-5), agrupados por País:")
print(descriptive_by_country_melted.to_markdown(index=False))
print(f"\nARCHIVO GENERADO (PARA REPOSITORIO): {csv_file_name}")

--- DIAGNÓSTICO AVANZADO: PROMEDIOS POR PAÍS ---
Tabla de Promedios por Indicador (en escala 1-5), agrupados por País:
| pais     | Indicador                     |   Promedio |
|:---------|:------------------------------|-----------:|
| Colombia | COMPETENCIAS_IA_Promedio      |      2.985 |
| Ecuador  | COMPETENCIAS_IA_Promedio      |      2.936 |
| México   | COMPETENCIAS_IA_Promedio      |      2.67  |
| Colombia | PERTINENCIA_LABORAL_Promedio  |      3.271 |
| Ecuador  | PERTINENCIA_LABORAL_Promedio  |      3.638 |
| México   | PERTINENCIA_LABORAL_Promedio  |      3.248 |
| Colombia | CONFIANZA_LABORAL_Promedio    |      3     |
| Ecuador  | CONFIANZA_LABORAL_Promedio    |      3     |
| México   | CONFIANZA_LABORAL_Promedio    |      3     |
| Colombia | IMPORTANCIA_FACTORES_Promedio |      4     |
| Ecuador  | IMPORTANCIA_FACTORES_Promedio |      3.889 |
| México   | IMPORTANCIA_FACTORES_Promedio |      3.599 |
| Colombia | EVALUACIÓN_DOCENTE_Promedio   |      3.308 |
| Ecuador  

In [7]:
# (Paso 6 - Celda 6) PROTOTIPO DE VISUALIZACIÓN BI (GRÁFICO)

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # Importamos pandas para cargar el archivo

# Cargamos el archivo generado en el paso anterior (Necesario si ejecutas la celda por separado)
csv_file_name = "analisis_promedios_por_pais.csv"
df_plot = pd.read_csv(csv_file_name)

# Excluir el indicador de Confianza Laboral (valor 3.0 constante) para mejor contraste visual
df_plot = df_plot[df_plot['Indicador'] != 'CONFIANZA_LABORAL_Promedio']

# Mapear nombres de indicadores a nombres más cortos y legibles
indicator_map = {
    'COMPETENCIAS_IA_Promedio': 'Competencias Digitales/IA',
    'PERTINENCIA_LABORAL_Promedio': 'Pertinencia Curricular',
    'IMPORTANCIA_FACTORES_Promedio': 'Importancia Factores Laborales',
    'EVALUACIÓN_DOCENTE_Promedio': 'Evaluación Universidad/Recursos',
    'ROL_PROFESORADO_Promedio': 'Rol Docente (Futuro)',
}
df_plot['Indicador_Corto'] = df_plot['Indicador'].map(indicator_map)

# Configuración y creación del gráfico
sns.set_theme(style="whitegrid")
plt.figure(figsize=(14, 8))
barplot = sns.barplot(
    data=df_plot,
    x='Promedio',
    y='Indicador_Corto',
    hue='pais',
    palette='viridis',
    orient='h'
)
# Línea de referencia para el nivel Neutral
plt.axvline(3.0, color='red', linestyle='--', linewidth=1.5, label='Nivel Neutral (3.0)')

plt.title('Promedio de Percepción por Indicador y País (Escala Likert 1-5)', fontsize=16)
plt.xlabel('Promedio de Percepción', fontsize=12)
plt.ylabel('Indicador de Evaluación', fontsize=12)
plt.xlim(1.0, 5.0)
plt.legend(title='País', loc='lower right')

# Añadir valores a las barras
for container in barplot.containers:
    barplot.bar_label(container, fmt='%.3f', padding=3, fontsize=9)

plt.tight_layout()

# Guardar la imagen (ENTREGABLE CLAVE para el Repositorio)
plt.savefig('prototipo_visualizacion_promedios.png')
plt.close()

print("\n--- VISUALIZACIÓN GENERADA! ---")
print("El archivo 'prototipo_visualizacion_promedios.png' ha sido guardado en tu carpeta.")

Matplotlib is building the font cache; this may take a moment.



--- VISUALIZACIÓN GENERADA! ---
El archivo 'prototipo_visualizacion_promedios.png' ha sido guardado en tu carpeta.


In [10]:
# (Celda 9) EXPORTAR EL DATAFRAME COMPLETO Y LIMPIO

# Usamos el DataFrame df_final que contiene todas las columnas *_num y las categóricas limpias
csv_limpio_final_name = "df_final_limpio_completo.csv"
df_final.to_csv(csv_limpio_final_name, index=False, sep=';')

print(f"ARCHIVO EXPORTADO: {csv_limpio_final_name}")
print("¡Sube este archivo a Google Drive para hacer los gráficos de frecuencia y correlación!")

ARCHIVO EXPORTADO: df_final_limpio_completo.csv
¡Sube este archivo a Google Drive para hacer los gráficos de frecuencia y correlación!


In [11]:
# (Celda 8) DISTRIBUCIÓN DEMOGRÁFICA POR GÉNERO

import matplotlib.pyplot as plt

# Calcular frecuencias
gender_counts = df_final['genero'].value_counts()
labels = gender_counts.index.tolist()
sizes = gender_counts.values.tolist()

plt.figure(figsize=(8, 8))

# Crea el gráfico de pastel
plt.pie(
    sizes, 
    labels=labels, 
    autopct='%1.1f%%', # Muestra el porcentaje con un decimal
    startangle=90,
    textprops={'fontsize': 10},
    wedgeprops={'edgecolor': 'black', 'linewidth': 0.5}
)

plt.title('Distribución Porcentual por Género', fontsize=14)
plt.axis('equal') # Asegura que el pastel sea un círculo
plt.tight_layout()
plt.savefig('grafico_demografico_genero.png')
plt.close()

print("\n--- GRÁFICO DE GÉNERO GENERADO! ---")


--- GRÁFICO DE GÉNERO GENERADO! ---


In [12]:
# (Celda 7) GRÁFICO DE DISPERSIÓN (CORRELACIÓN AVANZADA)

import matplotlib.pyplot as plt
import seaborn as sns

# Usamos las variables Promedio calculadas en el Paso 5
x_var = 'COMPETENCIAS_IA_Promedio'
y_var = 'PERTINENCIA_LABORAL_Promedio'

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 7))

# Crea el gráfico de dispersión, coloreado por país
scatter = sns.scatterplot(
    data=df_final,
    x=x_var,
    y=y_var,
    hue='pais',
    palette='deep',
    s=100, # Tamaño de los puntos
    alpha=0.6 # Transparencia
)

plt.title('Correlación entre Competencias Digitales y Pertinencia Laboral', fontsize=14)
plt.xlabel('Promedio de Competencias Digitales (Escala 1-4)', fontsize=12)
plt.ylabel('Promedio de Pertinencia Laboral (Escala 1-5)', fontsize=12)
plt.legend(title='País', loc='upper left')

plt.tight_layout()
plt.savefig('grafico_correlacion_comp_vs_pert.png')
plt.close()

print("--- GRÁFICO DE CORRELACIÓN GENERADO! ---")

--- GRÁFICO DE CORRELACIÓN GENERADO! ---


In [14]:
# (Celda 8) DISTRIBUCIÓN DEMOGRÁFICA POR GÉNERO

import matplotlib.pyplot as plt
import pandas as pd

# Calcular frecuencias
gender_counts = df_final['genero'].value_counts()
labels = gender_counts.index.tolist()
sizes = gender_counts.values.tolist()

plt.figure(figsize=(8, 8))

# Crea el gráfico de pastel
plt.pie(
    sizes, 
    labels=labels, 
    autopct='%1.1f%%', # Muestra el porcentaje con un decimal
    startangle=90,
    textprops={'fontsize': 10},
    wedgeprops={'edgecolor': 'black', 'linewidth': 0.5}
)

plt.title('Distribución Porcentual por Género', fontsize=14)
plt.axis('equal') # Asegura que el pastel sea un círculo
plt.tight_layout()
plt.savefig('grafico_demografico_genero.png')
plt.close()

print("--- GRÁFICO DE GÉNERO GENERADO! ---")

--- GRÁFICO DE GÉNERO GENERADO! ---


In [16]:
# (Paso 9 - Celda 9) STACKED BAR CHART: DISTRIBUCIÓN DE NIVELES DE COMPETENCIA (CORREGIDO)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Definición del mapeo numérico a etiquetas para la leyenda
level_map = {
    1: 'Ninguno', 
    2: 'Básico', 
    3: 'Intermedio', 
    4: 'Avanzado'
}

# 1. Crear una nueva columna categórica temporal a partir de la numérica
# ¡Usamos la columna que SÍ existe en df_final!
df_final['Nivel_Competencia'] = df_final['comp_manejo_ia_num'].map(level_map)

# 2. Definir el orden correcto de las etiquetas
order = ['Ninguno', 'Básico', 'Intermedio', 'Avanzado']

# 3. Crear una tabla de contingencia (frecuencias cruzadas)
cross_tab = pd.crosstab(
    df_final['pais'], 
    df_final['Nivel_Competencia'] # Usamos la nueva columna categórica
)

# 4. Normalizar por fila (por país) y reindexar para ordenar las barras
cross_tab_percent = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100
cross_tab_percent = cross_tab_percent.reindex(columns=order, fill_value=0) # Ordena las columnas

# --- 5. Crear la Visualización ---
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 7))

# Crea el gráfico de barras apiladas
cross_tab_percent.plot(
    kind='bar', 
    stacked=True, 
    colormap='viridis', 
    ax=plt.gca()
)

# Ajustes de estilo
plt.title('Distribución Porcentual del Dominio de Herramientas de IA por País', fontsize=14)
plt.xlabel('País', fontsize=12)
plt.ylabel('Porcentaje (%)', fontsize=12)
plt.xticks(rotation=0)

# Coloca la leyenda fuera del gráfico
plt.legend(title='Nivel de Dominio', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('grafico_competencias_stacked.png')
plt.close()

print("--- GRÁFICO DE BARRAS APILADAS GENERADO CON ÉXITO! ---")

--- GRÁFICO DE BARRAS APILADAS GENERADO CON ÉXITO! ---


In [17]:
# (Paso 10 - Celda 10) GRÁFICO DE ANÁLISIS DE BRECHA (IMPORTANCIA VS. REALIDAD)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el archivo de promedios para asegurar los datos más recientes
csv_file_name = "analisis_promedios_por_pais.csv"
df_gap = pd.read_csv(csv_file_name)

# 1. Pivotar la tabla para tener las dos métricas como columnas separadas
df_pivot = df_gap[
    df_gap['Indicador'].isin(['IMPORTANCIA_FACTORES_Promedio', 'COMPETENCIAS_IA_Promedio'])
].pivot(index='pais', columns='Indicador', values='Promedio').reset_index()

# 2. Calcular la brecha (Gap) para el título
df_pivot['Gap'] = df_pivot['IMPORTANCIA_FACTORES_Promedio'] - df_pivot['COMPETENCIAS_IA_Promedio']

# 3. Crear la Visualización (Dumbbell Plot simplificado)
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))

# Dibujar la línea que une Importancia y Competencia
plt.hlines(
    y=df_pivot['pais'], 
    xmin=df_pivot['COMPETENCIAS_IA_Promedio'], 
    xmax=df_pivot['IMPORTANCIA_FACTORES_Promedio'], 
    color='gray', 
    linewidth=4,
    alpha=0.6
)

# Dibujar los puntos de Importancia (Alto)
plt.scatter(
    df_pivot['IMPORTANCIA_FACTORES_Promedio'], 
    df_pivot['pais'], 
    color='#4CAF50', # Verde para el valor más alto (Importancia)
    s=200, 
    label='Importancia Percibida'
)

# Dibujar los puntos de Competencia (Bajo)
plt.scatter(
    df_pivot['COMPETENCIAS_IA_Promedio'], 
    df_pivot['pais'], 
    color='#F44336', # Rojo para el valor más bajo (Realidad)
    s=200, 
    label='Dominio Actual (Realidad)'
)

# Añadir etiquetas de valor
for index, row in df_pivot.iterrows():
    plt.text(row['Gap'] + row['COMPETENCIAS_IA_Promedio'] + 0.05, row['pais'], f"Brecha: {row['Gap']:.2f}", ha='left', va='center', fontsize=10)

plt.title('Análisis de Brecha: Importancia vs. Dominio de Competencias Digitales', fontsize=14)
plt.xlabel('Promedio Likert (Escala 1-5)', fontsize=12)
plt.ylabel('País', fontsize=12)
plt.xlim(2.5, 4.3) 
plt.legend(loc='lower right')

plt.tight_layout()
plt.savefig('grafico_gap_analysis.png')
plt.close()

print("--- GRÁFICO DE ANÁLISIS DE BRECHA GENERADO! ---")

--- GRÁFICO DE ANÁLISIS DE BRECHA GENERADO! ---


In [18]:
# (Paso 11 - Celda 11) HISTOGRAMA: DISTRIBUCIÓN POR EDAD

import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))

# Crea el histograma usando la columna numérica de edad
sns.histplot(
    data=df_final, 
    x='edad_num', 
    bins=10, # Divide los datos en 10 intervalos
    kde=True, # Añade una línea de densidad de probabilidad
    color='purple'
)

# Añade una línea para la media (promedio) de edad global
media_edad = df_final['edad_num'].mean()
plt.axvline(media_edad, color='red', linestyle='--', linewidth=1.5, label=f'Media: {media_edad:.1f} años')

plt.title('Distribución de Frecuencia de Estudiantes por Edad', fontsize=14)
plt.xlabel('Edad (Años)', fontsize=12)
plt.ylabel('Frecuencia (Conteo)', fontsize=12)
plt.legend()
plt.xlim(15, 70) # Establece un límite razonable para el eje x

plt.tight_layout()
plt.savefig('grafico_distribucion_edad.png')
plt.close()

print("--- GRÁFICO DE DISTRIBUCIÓN POR EDAD GENERADO! ---")

--- GRÁFICO DE DISTRIBUCIÓN POR EDAD GENERADO! ---
